In [60]:
# !pip install pandas
# !pip install scikit-learn
# !pip install numpy 
# install the above packages if it is not installed in your environment
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

#READ DATA

In [61]:
data = pd.read_csv('test_Data.csv')
# inorder to read data by timestamp sequentially, I sort and group data by the 'timestamp_id'
sorted_data_ts = data.sort_values(by="timestamp_id")
grouped_data_ts = sorted_data.groupby('timestamp_id')
# print(grouped_data_ts.head(5))

#FUNCTION TO FUSE AND SAVE DATA

In [73]:

def fuse_sensor_data(grouped_data,filename,threshold=2.0):
    header_written = False
    for timestamp, group in grouped_data:
        # print(timestamp,group)
        coords = group[['x_position', 'y_position']].values
        #DBSCAN clusteres based on distance by fitting x and y coordinates or positions
        clustered_db = DBSCAN(eps=threshold_distance, min_samples=1, metric='euclidean').fit(coords)
        labels = clustered_db.labels_
        unique_labels = np.unique(labels)
        fuse_data =[]
        #processing the ids for the clusters
        for label in unique_labels:
            cluster_data = group[labels == label]
            datetime_values = pd.to_datetime(cluster_data['timestamp_id']).view(np.int64)
            avg_timestamp = pd.Timestamp(datetime_values.mean()).isoformat(timespec='milliseconds') + "Z"

            # print(avg_timestamp)


            known_unique_ids = cluster_data[cluster_data['unique_id'] != 0]['unique_id'].unique()
            if len(known_unique_ids) > 0:
                f_u_id = known_unique_ids[0]
            else:
                f_u_id = 0
        
            cluster_list = cluster_data[['x_position', 'y_position', 'sensor_id']].values.tolist()

            fuse_data.append({
                'f_timestamp': avg_timestamp,
                'f_id': np.random.randint(1, 1000000), 
                'cluster_data': cluster_list,
                'f_u_id': f_u_id
            })
        fuse_data_df = pd.DataFrame(fuse_data)
        fuse_data_df.to_csv(filename, mode='a', header=(not header_written), index=False)
        header_written = True



In [74]:
threshold_distance = 2.0 
filename = 'fused_file.csv'
fuse_sensor_data(grouped_data_ts,filename,threshold=threshold_distance)